In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.neighbors import NearestNeighbors


## Question 1

Suppose that a process can be modeled via linear regression: 

```python
W = np.random.normal(1, 0, (1000,))
X = W + np.random.normal(1, 0, (1000,)) 
Z = np.random.normal(1, 0, (1000,)) 
Y = X + Z + W + np.random.normal(1, 0, (1000,))
```

Which is closest to the correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿? 

In [12]:
np.random.seed(42)
W = np.random.normal(1, 0, (1000,))
X = W + np.random.normal(1, 0, (1000,))
Z = np.random.normal(1, 0, (1000,))
Y = X + Z + W + np.random.normal(1, 0, (1000,))

In [13]:
df = sm.add_constant(pd.DataFrame({"X": X}))
results = sm.OLS(Y, df).fit()
results.bse

X    2.810072e-17
dtype: float64

In [18]:
#Calculate the variance of W
W_var = np.var(W)
W_var


0.0

## Question 2

If ﻿Y﻿ is written as depending on ﻿X﻿ and ﻿Z﻿ only, ﻿W﻿ is part of the error term. Which, then, is closest to the expected correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿? 

In [19]:
n = 10000

# Generate W, epsilon_X, epsilon_Y, and Z
W = np.random.normal(0, 1, n)
epsilon_X = np.random.normal(0, 1, n)
epsilon_Y = np.random.normal(0, 1, n)
Z = np.random.normal(0, 1, n)

# Generate X and Y
X = W + epsilon_X
Y = X + Z + W + epsilon_Y  # True model includes W

# Regress Y on X and Z (omitting W)
X_Z = np.column_stack((X, Z))
X_Z = sm.add_constant(X_Z)  # Add intercept
model = sm.OLS(Y, X_Z).fit()

# Get residuals (error term)
residuals = model.resid

# Compute correlation between X and the error term
correlation = np.corrcoef(X, residuals)[0, 1]

print(f"Correlation between X and residuals (error): {correlation:.4f}")

Correlation between X and residuals (error): 0.0000


## Question 3

In the data frame for homework_7.1.csv, control for W by regressing ﻿Y﻿ on ﻿X﻿ and ﻿Z﻿ at the following constant values of ﻿W﻿: -1, 0, and 1. (You cannot literally use a constant value of ﻿W﻿, of course, or you will have only one data point! How will you manage this?) The question is: Is the coefficient of ﻿X﻿  

In [20]:
df = pd.read_csv("homework_7.1.csv")  # Replace with your actual file path or string

# Define target W values and tolerance
target_W_values = [-1, 0, 1]
tolerance = 0.1

results = []

for w_val in target_W_values:
    # Subset the data for W near the target value
    df_subset = df[(df["W"] >= w_val - tolerance) & (df["W"] <= w_val + tolerance)]
    
    # Prepare regressors and response
    X_vars = sm.add_constant(df_subset[["X", "Z"]])
    y_var = df_subset["Y"]

    # Run OLS regression
    model = sm.OLS(y_var, X_vars).fit()

    # Store the coefficient of X
    results.append({
        "W approx": w_val,
        "n": len(df_subset),
        "X coeff": model.params["X"],
        "p-value": model.pvalues["X"],
        "R²": model.rsquared
    })

# Display results
results_df = pd.DataFrame(results)
print(results_df)

   W approx    n   X coeff        p-value        R²
0        -1  488  0.857978   1.921114e-59  0.509544
1         0  780  1.383211  9.340880e-188  0.697200
2         1  455  1.958097  1.297296e-164  0.817845


## Question 4

```python
def make_error(corr_const, num): 
 err = list() 
 prev = np.random.normal(0, 1) 
 for n in range(num): 
    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1) 
    err.append(prev) 
 return np.array(err) 
```


Create a linear regression model that uses this function as the error for both (a) the treatment, ﻿X﻿, and (b) the outcome, ﻿Y﻿. (You can use random normal error for any other covariates, if you have them.) 
As corr_const increases from 0.2 to 0.5 to 0.8, find (i) the standard deviation of the estimate of the ﻿X﻿ coefficient over many trials, and (ii) the mean of the standard error estimate of the ﻿X﻿ coefficient over many trials. 
When corr_const increases, then: 

Hint: don't forget to include an intercept in your regression

In [21]:
def make_error(corr_const, num): 
 err = list() 
 prev = np.random.normal(0, 1) 
 for n in range(num): 
    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1) 
    err.append(prev) 
 return np.array(err) 

In [22]:
def run_simulation(corr_const, num_trials=1000, n=100):
    beta_ests = []
    se_ests = []

    for _ in range(num_trials):
        e_X = make_error(corr_const, n)
        e_Y = make_error(corr_const, n)

        X = 1 + e_X  # Include intercept-like component
        Y = 2 + 3 * X + e_Y  # True model: Y = 2 + 3X + noise

        # Regression
        X_design = sm.add_constant(X)
        model = sm.OLS(Y, X_design).fit()
        
        beta_ests.append(model.params[1])  # Coefficient for X
        se_ests.append(model.bse[1])       # Standard error of X coefficient

    # Compute summary statistics
    std_beta = np.std(beta_ests)
    mean_se = np.mean(se_ests)

    return std_beta, mean_se

# Run for different correlation constants
results = []

for corr in [0.2, 0.5, 0.8]:
    std_b, mean_se = run_simulation(corr_const=corr)
    results.append({
        "corr_const": corr,
        "std_dev_beta_hat": std_b,
        "mean_std_error": mean_se
    })

# Show results
df_results = pd.DataFrame(results)
print(df_results)

   corr_const  std_dev_beta_hat  mean_std_error
0         0.2          0.106568        0.101308
1         0.5          0.133763        0.100808
2         0.8          0.239446        0.100172


## Reflection Questions

1. Create a linear regression model involving a confounder that is left out of the model.  Show whether the true correlation between $X$ and $Y$ is overestimated, underestimated, or neither.  Explain in words why this is the case for the given coefficients you have chosen.



In [2]:
# Set seed for reproducibility
np.random.seed(0)

# Sample size
n = 1000

# Generate W
W = np.random.normal(0, 1, n)

# Make X correlated with W
X = 0.7 * W + np.sqrt(1 - 0.7**2) * np.random.normal(0, 1, n)

# True coefficients
beta_0 = 0
beta_1 = 2  # effect of X
beta_2 = 3  # effect of W

# Generate Y
epsilon = np.random.normal(0, 1, n)
Y = beta_0 + beta_1 * X + beta_2 * W + epsilon

# Regress Y on X (misspecified, omitting W)
X_with_const = sm.add_constant(X)
model_misspec = sm.OLS(Y, X_with_const).fit()
print("Misspecified model (without W):")
print(model_misspec.summary())

# Regress Y on X and W (correct)
XW_with_const = sm.add_constant(np.column_stack((X, W)))
model_correct = sm.OLS(Y, XW_with_const).fit()
print("\nCorrect model (with W):")
print(model_correct.summary())


Misspecified model (without W):
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     2932.
Date:                Sat, 12 Jul 2025   Prob (F-statistic):          2.61e-299
Time:                        10:56:24   Log-Likelihood:                -2255.6
No. Observations:                1000   AIC:                             4515.
Df Residuals:                     998   BIC:                             4525.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.140

In this example, 𝑋 and 𝑊 are positively correlated, and both increase 𝑌. When you leave out 𝑊, the model attributes part of 𝑊’s influence on 𝑌 to 𝑋, overestimating the effect of 𝑋.

If instead:

- 𝛽2 < 0 (negative effect of 𝑊 on 𝑌), or
- corr(𝑋, 𝑊) < 0 (negative correlation),

then the bias direction could flip, leading to underestimation.



2. Perform a linear regression analysis in which one of the coefficients is zero, e.g.

W = [noise]
X = [noise]
Y = 2 * X + [noise]

And compute the p-value of a coefficient - in this case, the coefficient of W.  
(This is the likelihood that the estimated coefficient would be as high or low as it is, given that the actual coefficient is zero.)
If the p-value is less than 0.05, this ordinarily means that we judge the coefficient to be nonzero (incorrectly, in this case.)
Run the analysis 1000 times and report the best (smallest) p-value.  
If the p-value is less than 0.05, does this mean the coefficient actually is nonzero?  What is the problem with repeating the analysis?

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

# Parameters
n_samples = 100         # number of data points in each simulation
n_simulations = 1000    # number of repeated simulations

# Store minimum p-value found across simulations
min_p_value = 1.0

# Store all p-values for summary
all_p_values = []

# Run in a loop
for _ in range(n_simulations):
    # Generate random noise variables
    W = np.random.normal(0, 1, n_samples)
    X = np.random.normal(0, 1, n_samples)
    noise = np.random.normal(0, 1, n_samples)
    
    # Generate Y where only X matters (coefficient for W is truly zero)
    Y = 2 * X + noise

    # Create design matrix with intercept, W, and X
    X_design = sm.add_constant(np.column_stack((W, X)))
    
    # Fit linear regression model
    model = sm.OLS(Y, X_design).fit()
    
    # Get p-value for W (index 1: [const, W, X])
    p_value_W = model.pvalues[1]
    all_p_values.append(p_value_W)
    
    # Update minimum p-value if smaller one is found
    if p_value_W < min_p_value:
        min_p_value = p_value_W

# Convert p-values to DataFrame for summary or export
summary_df = pd.DataFrame({'p_value_W': all_p_values})

# Count how many times p-value was below 0.05
significant_count = (summary_df['p_value_W'] < 0.05).sum()

# Report results
print(f"Minimum p-value found: {min_p_value}")
print(f"Number of runs with p-value < 0.05: {significant_count} out of {n_simulations}")

# Optional: show p-value summary table
print(summary_df.describe())


Minimum p-value found: 0.0004745704124689032
Number of runs with p-value < 0.05: 51 out of 1000
         p_value_W
count  1000.000000
mean      0.495585
std       0.286279
min       0.000475
25%       0.244707
50%       0.504390
75%       0.739878
max       0.999762


A p-value below 0.05 does not guarantee the coefficient is nonzero. It just means that, if the true coefficient were zero, we’d observe such an extreme estimate about 5% of the time by random chance.

By running the analysis 1,000 times and picking the smallest p-value, you artificially inflate the chance of finding a “significant” result this is called p-hacking. The more tests you run, the more likely you are to get a small p-value by chance, even when no true effect exists. So, finding a p-value < 0.05 after many repeated trials does not imply a real effect. It often just reflects statistical noise.
